In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def reset_tf():
    global sess
    sess.close()
    tf.reset_default_graph()
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [3]:
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [191]:
reset_tf()

seq_length = 12
batch_size = 1
# embedding_size = 64
hidden_size = 12
vocab_size = 12

input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
# TODO: fill in sequence_length as per 
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
input_lengths = tf.placeholder(tf.int32, [batch_size])
target_data = tf.placeholder(tf.int32, [batch_size, seq_length])

rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_size)

initial_states = rnn_cell.zero_state(batch_size, tf.float32)

embedded_inputs = tf.one_hot(input_data, vocab_size)

# embedding = tf.get_variable('embedding', [vocab_size, embedding_size])
# embedded_inputs = tf.nn.embedding_lookup(embedding, input_data)

softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size])
softmax_b = tf.get_variable("softmax_b", [vocab_size])

outputs, final_states = tf.nn.dynamic_rnn(rnn_cell,
                                          embedded_inputs, 
                                          initial_state=initial_states, 
                                          sequence_length=input_lengths)

flat_outputs = tf.reshape(outputs, [-1, hidden_size])
flat_targets = tf.reshape(target_data, [-1])

flat_output_logits = tf.matmul(flat_outputs, softmax_w) + softmax_b
flat_output_probs = tf.nn.softmax(flat_output_logits)

# flat_loss_mask = tf.sign(tf.to_float(flat_targets))
# flat_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flat_output_logits, labels=flat_targets) * flat_loss_mask
flat_losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flat_output_logits, labels=flat_targets)

mean_loss = tf.reduce_mean(flat_losses)
total_loss = tf.reduce_sum(flat_losses)

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# gvs = optimizer.compute_gradients(mean_loss)
# capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
# train_op = optimizer.apply_gradients(capped_gvs)

train_op = tf.train.AdamOptimizer(0.1).minimize(mean_loss)

sess.run(tf.global_variables_initializer())

In [5]:
train_text = """the sun did not shine.
it was too wet to play.
so we sat in the house
all that cold, cold, wet day.

i sat there with sally.
we sat there, we two.
and i said, 'how i wish
we had something to do!'

too wet to go out
and too cold to play ball.
so we sat in the house.
we did nothing at all."""

In [133]:
def generate_batches(array, batch_size, seq_length):
    num_seqs = (len(array) + seq_length - 1) // seq_length
    num_seqs_per_batch = (num_seqs + batch_size - 1) // batch_size
    
    for i in range(num_seqs_per_batch):
        seqs = []
        seq_lens = []
        
        for j in range(batch_size):
            offset = (j*num_seqs_per_batch + i)*seq_length
            
            seq = array[offset:offset+seq_length]
            seq_len = len(seq)
            seq = np.pad(seq, (0,seq_length-len(seq)), 'constant', constant_values=0)
            
            seqs.append(seq)
            seq_lens.append(seq_len)
            
        yield np.stack(seqs), seq_lens
        
    return

In [134]:
def sample(initial_input, count):
    curr_initial_states  = np.zeros(initial_states.shape)
    curr_input_data = np.zeros(input_data.shape)
    curr_input_lengths = [1] + [0] * (batch_size - 1)
    
    result = []
    
    for i in range(count):
        ps, curr_initial_states = sess.run((flat_output_probs, final_states), feed_dict = {
            input_data: curr_input_data,
            input_lengths: curr_input_lengths,
            initial_states: curr_initial_states
        })
        result.append(np.random.choice(len(ps[0]), p = ps[0]))
    
    return result

In [136]:
train_array = np.array([ord(ch) for ch in train_text])

for i in range(100):
    epoch_loss = 0.0
    curr_initial_states = np.zeros(initial_states.shape)
    train_input_batches = generate_batches(train_array[:-1], batch_size, seq_length)
    train_target_batches = generate_batches(train_array[1:], batch_size, seq_length)
    
    for (curr_input_data, curr_input_lens), (curr_target_data, _) in zip(train_input_batches, train_target_batches):
        feed_dict = {
            input_data: curr_input_data, 
            input_lengths: curr_input_lens,
            target_data: curr_target_data,
            initial_states: curr_initial_states }
        _, curr_loss, curr_initial_states = sess.run((train_op, total_loss, final_states), feed_dict = feed_dict)
        epoch_loss += curr_loss
        
    epoch_loss /= len(train_array) - 1
    
    if i % 10 == 0:
        print(f'epoch {i}: loss={epoch_loss}')


epoch 0: loss=1.3683575254699494
epoch 10: loss=1.1983166629844426
epoch 20: loss=1.2701033632099006
epoch 30: loss=1.8854995571362432
epoch 40: loss=2.0423659314677276
epoch 50: loss=1.9313351169280475
epoch 60: loss=1.669676526498296
epoch 70: loss=1.2333975004402187
epoch 80: loss=1.4025340894373453
epoch 90: loss=1.5118542265808956


In [194]:
curr_initial_states = np.zeros(initial_states.shape)
curr_input_data = [list(range(12))]
curr_target_data = [list(range(1,13))]
curr_input_lengths = [12]

feed_dict = {
    input_data: curr_input_data, 
    input_lengths: curr_input_lens,
    target_data: curr_target_data,
    initial_states: curr_initial_states
}

# for i in range(500):
#     _, curr_loss = sess.run((train_op, total_loss), feed_dict = feed_dict)
#     if i % 50:
#         print(curr_loss)

# print(curr_input_data)
# print(curr_target_data)
#np.argmax(sess.run(flat_output_probs, feed_dict = feed_dict), axis=1)

sess.run(outputs, feed_dict = feed_dict)



array([[[-0.33371767,  0.22937587, -0.04559825, -0.03681822, -0.10696881,
          0.06150694,  0.33818346, -0.33722278,  0.27803215, -0.310305  ,
         -0.34002498,  0.0376387 ],
        [ 0.34987506, -0.5340544 , -0.22623123, -0.08637503, -0.3168208 ,
         -0.23286073, -0.47730848, -0.13026679,  0.14726035,  0.15282296,
         -0.18769382, -0.41964367],
        [-0.02668606,  0.00458021,  0.00452762,  0.02079288, -0.11766426,
          0.10440546, -0.51964724,  0.5665893 , -0.32957262, -0.01001556,
         -0.60038722,  0.45001325],
        [-0.03273053,  0.14313154,  0.26246089,  0.42619997,  0.37390828,
          0.15373884, -0.55407733,  0.63674074,  0.34912387,  0.03527905,
         -0.07635973, -0.02045194],
        [ 0.051635  , -0.02781652, -0.07707702, -0.25481865,  0.55797064,
         -0.22971286,  0.24407208, -0.16768903, -0.49062645,  0.34846655,
         -0.17940263, -0.01900988],
        [ 0.29846254,  0.03369355, -0.09964083,  0.14160274,  0.19294849,
      